# Kaggle Study Day 21

# Fruits-360 - Transfer Learning using Keras

https://www.kaggle.com/amadeus1996/fruits-360-transfer-learning-using-keras

-- 내 정리 - 시온 -- <br>
Transfer Learning : 전이학습
- 이미지 분류에 사용
- 비교적 짧은 시간에 높은 정확도 달성
- 주로 사전학습된 모델(pre-trained) 이용 : 사전학습모델이란, 풀고자 하는 문제와 비슷하며 대용량 데이터로 이미 학습되어있는 모델. 이것을 import 해서 사용 (예: VGG, MobileNet, Inception)
- 데이터가 부족한(데이터셋이 작은) 분야에 적용 가능

- Fruits-360 데이터셋에 전이학습을 간단히 적용한 커널이다.
- 데이터셋은 64개 과일 이미지 42345개로 구성된다.
- 전이학습 접근방식을 일반 접근방식과 비교한다.
- 98.44%의 정확도는 2 epochs 내에서 얻어진다.

1. 전이학습에 대한 간략한 설명
2. Kaggle Kernel을 사용한 전이학습
3. 데이터 읽 및 시각화
4. 모델 구축 및 컴파일
5. 사전학습(pre-trained) 모델 학습 및 검증
6. Vanilla 모델 학습 및 검증
7. 사전학습모델과 Vanilla 모델 비교

In [2]:
import os
from os import listdir, makedirs
from os.path import join, exists, expanduser

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as k
import tensorflow as tf

# 1. 전이학습

전이학습에서 먼저 기본 데이터셋으로 기본 네트워크를 학습시키고 나서, target 데이터셋과 작업에 대해 학습되도록 학습된 변수를 변경하거나, 두번째 target 네트워크에 학습된 변수를 전달(전이)한다. 이 과정은 기본 작업에 특정되는 대신, 기본 작업과 target 작업 모두에 적합하게 일반적일 경우 작동하는 경향이 있다.

Lisa Torrey와 Jude Shavlik은 전이학습을 사용할 경우의 세가지 이점을 설명한다.
- Higher start : 원래 모델에서 초기 기술(모델을 다듬기 전)이 그렇지 않은 경우보다 높다.
- Higher slope : 원래 모델의 학습 도중 스킬 향상 속도는 그렇지 않은 경우보다 더 가파르다.
- Higher asymptote : 학습된 모델의 융합된 기술은 그렇지 않은 경우보다 더 우수하다.

기본적으로, 사전학습된 모델(다른 누군가에 의해 대규모 데이터셋을 학습한 네트워크의 가중치와 파라미터)을 취하고 자체 데이터셋으로 모델을 "fine-tune"한다. 이는 사전학습된 모델이 초기 가중치를 제공해 더 빠른 수렴을 유도하거나, 관심 작업의 고정 변수 추출기로 작용할 것이라는 아이디어다.

이러한 두 가지 주요 전이학습 시나리오는 다음과 같다.
- Finetunning the convnet : 랜덤초기화 대신, imagenet 1000처럼 대규모 데이터셋을 학습한 것과 같이, 사전학습된 네트워크로 네트워크를 초기화한다. 나머지 train 데이터는 평소와 같다. 이 시나리오에서 전체 네트워크는 관심 데이터셋에 대해 재학습해야한다.
- ConvNet as fixed feature extractor : 여기서, 최종적으로 완전연결된 레이어(Dense)의 가중치를 제외한 네트워크의 모든 가중치를 고정시킨다. 마지막으로 완전연결된 레이어는 랜덤 가중치를 가진 새로운 레이어와 대체되고 이 레이어만 학습된다.

이 커널에선 첫 시나리오를 실행한다.

# 2. Transfer Learning using Kaggle Kernels

#### Using the Keras Pretrained Models dataset

캐글 커널은 사전학습된 keras 모델을 다운로드하기 위해 네트워크 연결을 사용할 수 없다. 이 데이터셋은 캐글 커널 환경에서 우리가 선호하는 사전학습된 모델을 사용할 수 있도록 도와준다.

해야할 일은 kears가 찾고있는 캐시 디렉토리(~/.keras/models)에 사전학습된 모델을 복사하는 것이다.

In [6]:
cache_dir = expanduser(join('~', 'keras'))
if not exists(cache_dir):
    makedirs(cache_dir)
models_dir = join(cache_dir, 'models')
if not exists(models_dir):
    makedirs(models_dir)

# 3. Reading and Visualizing the Data

## Reading the Data

남은 Keras처럼, 이미지 증강 API는 간단하고 강력하다. ImageDataGenrator를 사용해 데이터를 가져오고 네트워크에 제공할 것이다.

Keras는 이미지 데이터 준비와 증강을 위한 구성을 정의하는 ImageDataGenerator 클래스를 제공한다. 메모리에서 전체 이미지 데이터셋에 대한 작업을 수행하는 것보다, 딥러닝 모델 fit 과정을 통해 반복되도록 API를 설계하여 제때에 증강 이미지 데이터를 생성한다. 이렇게 하면 메모리 오버헤드가 줄어들지만, 모델 학습 시간이 약간 늘어난다.

데이터 생성기 자체는 사실 반복자(iterator)라서 요청시 디렉토리에서 이미지 샘플의 batch를 반환한다. flow_from_directory() 함수를 호출해 batch size를 구성하고 데이터 생성기를 준비하고 이미지의 batch를 얻을 수 있다.

In [8]:
# 이미지의 차원
img_width, img_height = 224, 224  
# 사용할 사전학습된 모델에 따라 img_width, img_height 설정
# ResNet-50의 input size는 224 * 224 * 3이다.

train_data_dir = '../input/fruits/fruits-360_dataset_2018_06_03/fruits-360/Training'
validation_data_dir = '../input/fruits/fruits-360_dataset_2018_06_03/fruits-360/Validation/'
nb_train_samples = 31688
nb_validation_samples = 10657
batch_size = 16

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir, target_size=(img_height, img_width), batch_size=batch_size, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir, target_size=(img_height, img_width), batch_size=batch_size, class_mode='categorical')

## Visualizing the Data

In [9]:
import pandas as pd
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [ ]:
training_data = pd.DataFrame(train_generator.classes, columns=['classes'])
testing_data = pd.DataFrame(validation_generator.classes, columns=['classes'])

In [10]:
def create_stack_bar_data(col, df):
    aggregated = df[col].value_counst().sort_index()
    x_values = aggregated.index.tolist()
    y_values = aggregated.values.tolist()
    return x_values, y_values

In [ ]:
x1, y1 = create_stack_bar_data('classes', training_data)
x1 = list(train_generator.class_indices.keys())

trace1 = go.Bar(x=x1, y=y1, opacity=0.75, name='Class Count')
layout = dict(height=400, width=1200, title='Class Distribution in Training Data',
             legend=dict(orientation='h'), yaxis=dict(title='Class Count'))
fig = go.Figure(data=[trace1], layout=layout)
iplot(fig)

In [ ]:
x1, y1 = create_stack_bar_data('classes', testing_data)
x1 = list(validation_generator.class_indices.keys())

trace1 = go.Bar(x=x1, y=y1, opacity=0.75, name='Class Count')
layout = dict(height=400, width=1100, title='Class Distribution in Validation Data',
             legend=dict(orientation='h'), yaxis=dict(title='Class Count'))
fig = go.Figure(data=[trace1], layout=layout)
iplot(fig)

볼 수 있듯이 모든 클래스는 validation 뿐만 아니라 training에서도 균형이 잘 잡혀있다.

## Building the Models

여기서는 ImageNet의 가중치로 ResNet-50을 로딩한다. 클래스의 수에 따라 자체 레이어를 추가할 수 있도록 탑을 제거한다. 그런 다음 모델 아키텍쳐를 완성하기 위해 자체 레이어를 추가한다.

### Building Pretrained Model

In [ ]:
# 사전학습된 가중치로 inception을 import. 완전연결된 레이어는 포함하지 않는다.
inception_base = applications.ResNet50(weights='imagenet', include_top=False)

# global spatial average pooling 레이어 추가
x = inception_base.output
x = GlobalAveragePooling2D()(x)
# 완전연결된 레이어 추가
x = Dense(512, activation='relu')(x)
# 완전연결된 output과 분류 레이어 
predictions = Dense(65, activation='softmax')(x)
# 전체 네트워크를 생성해 학습시킬 수 있다
inception_transfer = Model(inputs=inception_base.input, outputs=prediction)

### Building the Vanilla Model

In [ ]:
# 사전학습된 가중치로 inception을 import. 완전연결된 레이어는 포함하지 않는다.
inception_base_vanilla = applications.ResNet50(weights=None, include_top=False)

# global spatial average pooling 레이어 추가
x = inception_base_vanilla.output
x = GlobalAveragePooling2D()(x)
# 완전연결된 레이어 추가
x = Dense(512, activation='relu')(x)
# 완전연결된 output과 분류 레이어 
predictions = Dense(54, activation='softmax')(x)
# 전체 네트워크를 생성해 학습시킬 수 있다.
inception_transfer_vanilla = Model(inputs=inception_base_vanilla.input, outputs=predictions)

## Compiling the Models

손실함수(loss)와 사용할 최적화 알고리즘, 각 epoch 마지막에 계산될 metrics를 설정한다.

In [ ]:
inception_transfer.compile(loss='categorical_crossentropy', 
                          optimizer=opimizers.SGD(lr=1e-4, momentum=0.9),
                           metrics=['accuracy'])
inception_transfer_vanilla.compile(loss='categorical_crossentropy',
                                  optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
                                  metrics=['accuracy'])

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())